# EDA

## ***선행작업***

In [ ]:
!pip install konlpy
!pip install collections
!pip install wordcloud
!pip install sweetviz


     |████████████████████████████████| 19.4 MB 438 kB/s 
     |████████████████████████████████| 448 kB 69.6 MB/s 
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
     |████████████████████████████████| 15.1 MB 4.1 MB/s 


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
import sweetviz as sv
import missingno as msno
# 다운받은 csv를 pandas의 DataFrame 형식으로 불러옵니다.
data = pd.read_csv('train.csv')

data[20:100] # 데이터를 확인합니다.

,id,document,label
20,21,"처절한 영화, 슬픈 영화, 씁쓸-한 영화...",1
21,22,처음부터 끝까지 유치 하기 그지 없다....난해한 영화,0
22,23,전쟁? 아이들의 목소리가 참 거슬린다.,0
23,24,멋진 상상력! 아름다운 영화. 여배우 소이의 재발견!,1
24,25,그 시대엔 이 영화가 최고였음,1
...,...,...,...
95,96,표 값과 시간이 아까웠던 영화,0
96,97,쩌는 시나리오 쩌는 연기 환상의 소피아 결론은 제이미 벨 이 축복받은 놈,1
97,98,너무 기분좋게봣습니다~ 적극 추천!!,1
98,99,왜 그랬니? 피트 출현할 영화가 그렇게 없디?,0


## ***데이터 분석하기***

## ***그냥 둘러보기***

In [ ]:
df_train = data.copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [ ]:
#결측값은 없는지 확인하기
for col in df_train.columns:
    msg = 'column: {:>20} \t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

column:                   id 	 Percent of NaN value: 0.00%
column:             document 	 Percent of NaN value: 0.00%
column:                label 	 Percent of NaN value: 0.00%


## ***(문장)끼리 비교해보기***

In [ ]:
val_count = data['label'].value_counts() # 유니크값의 개수를 확인합니다.
print('라벨 0인 리뷰 개수:', val_count[0])
print('라벨 1인 리뷰 개수:', val_count[1])

라벨 0인 리뷰 개수: 2564
라벨 1인 리뷰 개수: 2436


In [ ]:
len_1 = 0
len_0 = 0
for i in range(len(df_train)):
  str = df_train['document'][i]
  if df_train['label'][i] == 1:
    len_1 += len(str)
  if df_train['label'][i] == 0:
    len_0 += len(str)
  
print(f"긍정 평가 평균 길이 : {len_1/val_count[1]}\n부정 평가 평균 길이 : {len_0/val_count[0]}")

긍정 평가 평균 길이 : 27.188013136288998
부정 평가 평균 길이 : 27.085023400936038


##***(단어)의 통계적인 비율 살펴보기***

In [ ]:
bad_word = {}
bad_num = 0

for i in range(5000):
  if df_train['label'][i] == 0:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in bad_word:
        bad_word[i] = 1
        bad_num += 1
      else:
        bad_word[i] += 1
        bad_num += 1

# lambda x : x[1] 
d1 = sorted(bad_word.items(), key=lambda x: x[1], reverse=True) 
#print(d1)
for i in range(20):
  print(d1[i])
print(f"전체 악평 단어 개수 : {bad_num}")


('영화', 754)
('이', 143)
('진짜', 133)
('점', 130)
('평점', 119)
('왜', 116)
('재미', 105)
('정말', 104)
('스토리', 102)
('최악', 98)
('시간', 98)
('내용', 90)
('것', 89)
('뭐', 87)
('감독', 79)
('거', 78)
('내', 77)
('그냥', 77)
('돈', 75)
('연기', 73)
전체 악평 단어 개수 : 11854


In [ ]:
good_word = {}
good_num = 0

for i in range(5000):
  if df_train['label'][i] == 1:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in good_word:
        good_word[i] = 1
        good_num += 1
      else:
        good_word[i] += 1
        good_num += 1

# lambda x : x[1] 
d2 = sorted(good_word.items(), key=lambda x: x[1], reverse=True) 
for i in range(20):
  print(d2[i])
print(f"전체 호평 단어 개수 : {good_num}")

('영화', 769)
('정말', 216)
('최고', 214)
('진짜', 136)
('감동', 135)
('연기', 111)
('드라마', 90)
('보고', 89)
('때', 83)
('이', 79)
('사랑', 74)
('다시', 72)
('평점', 67)
('점', 67)
('것', 66)
('생각', 65)
('말', 63)
('명작', 61)
('내', 58)
('배우', 58)
전체 호평 단어 개수 : 10995


In [ ]:
print(f"부정 단어 가짓수 : {len(d1)}")
print(f"긍정 단어 가짓수 : {len(d2)}")

부정 단어 가짓수 : 3227
긍정 단어 가짓수 : 3110


In [ ]:
def persent(n, word_num, d):
  sum = 0
  for i in range(n):
    sum += d[i][1]/word_num
  print(f"단어 {n}개 사용했을 때 비율 : {round(sum*100, 2)} %")

In [ ]:
print("부정 단어")
for i in range(500, 2000, 200):
  persent(i, bad_num, d1)
print("\n긍정단어")
for i in range(500, 2000, 200):
  persent(i, good_num, d2)

부정 단어
단어 500개 사용했을 때 비율 : 68.23 %
단어 700개 사용했을 때 비율 : 73.59 %
단어 900개 사용했을 때 비율 : 77.42 %
단어 1100개 사용했을 때 비율 : 80.79 %
단어 1300개 사용했을 때 비율 : 83.74 %
단어 1500개 사용했을 때 비율 : 85.43 %
단어 1700개 사용했을 때 비율 : 87.12 %
단어 1900개 사용했을 때 비율 : 88.81 %

긍정단어
단어 500개 사용했을 때 비율 : 68.85 %
단어 700개 사용했을 때 비율 : 74.08 %
단어 900개 사용했을 때 비율 : 77.72 %
단어 1100개 사용했을 때 비율 : 81.36 %
단어 1300개 사용했을 때 비율 : 83.54 %
단어 1500개 사용했을 때 비율 : 85.36 %
단어 1700개 사용했을 때 비율 : 87.18 %
단어 1900개 사용했을 때 비율 : 88.99 %


### ***단어 목록 살펴보기 (전체 다 보면 수위가 높은 단어들도 많이보인다)***

In [ ]:
for i in range(40):
  print(d1[i][0])

In [ ]:
for i in range(40):
  print(d2[i][0])

# ***예측해보기***


In [ ]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data['document'].nunique(), train_data['label'].nunique() # 중복 없음

(5000, 2)

In [ ]:
#이상한 문자 지우기
import re
eng_text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
print(re.sub(r'[^a-zA-Z ]', '', eng_text))
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

do you expect people to read the FAQ etc and actually accept hard atheism


,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1
2,3,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1
3,4,아무래도 년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,0


In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "") 
# white space 데이터를 empty value로 변경

In [ ]:
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

id           0
document    10
label        0
dtype: int64


In [ ]:
train_data.loc[train_data.document.isnull()][:10]

,id,document,label
159,160,NaN,1
659,660,NaN,1
721,722,NaN,1
1921,1922,NaN,1
2466,2467,NaN,1
2717,2718,NaN,1
3693,3694,NaN,1
4126,4127,NaN,1
4147,4148,NaN,1
4765,4766,NaN,1


In [ ]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

4990


In [ ]:
test_data = pd.read_csv("test.csv") #파일 읽기
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))


전처리 후 테스트용 샘플의 개수 : 5000


In [ ]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', "꿀잼", inplace=True) # 공백은 Null 값으로 변경


print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 5000


## 모델 사용

In [ ]:
x = train_data.document #training 데이터에서 문서 추출
y = train_data.label #training 데이터에서 라벨 추출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #sklearn 패키지의 CountVectorizer import
CV = CountVectorizer() #countvectorizer 생성
CV.fit(x)
print(x[0:10])
x = CV.transform(x)
print(x[0:2])

0                영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
1                히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯
2         괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요
3                아무래도 년도지난작품이라 지금보기는너무유치하다
4    지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다
5        내인생에 있어 손까락에 꼽히는 영화 원작을 능가한 리메이크작
6                성룡영화중 최악  차라리 러시아워가 훨씬 나음
7              마음이 성숙하고 보면 다르게 보이는 영화 걸작이다
8                       가슴 찡했던 오래 기억에남을 영화
9                 결말 기분나쁘다 공포매니아에게 이따위 결말을
Name: document, dtype: object
  (0, 4972)	1
  (0, 5963)	1
  (0, 9072)	1
  (0, 10074)	1
  (0, 10341)	1
  (0, 14394)	1
  (1, 5054)	1
  (1, 5820)	1
  (1, 5925)	1
  (1, 9180)	1
  (1, 13031)	1
  (1, 14907)	1


In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(x, y) #모델 학습

LogisticRegression()

In [ ]:
#run model
X_pred = CV.transform(["영화 완전 노잼"]) 
y_pred = model.predict(X_pred)
print(y_pred)

[0]


# 제출

In [ ]:
test_data

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱
2,3,꿀잼
3,4,멋있는 영화입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔
4997,4998,지금까지 나온 드라마 중 최고였습니다 점 만점
4998,4999,영화값으로 엄청난 공연을 보았다 이게 웬 횡재


In [ ]:
test_X = test_data.document #문서 데이터 생성
vectorizer = CV
test_X_vect = vectorizer.transform(test_X) #문서 데이터 transform 
#test 데이터를 대상으로 fit_transform 메소드를 실행하는 것은 test 데이터를 활용해 vectorizer 를 학습 시키는 것으롤 data leakage 에 해당합니다.

pred = model.predict(test_X_vect) #test 데이터 예측

In [ ]:
submission = pd.read_csv("./sample_submission.csv") #제출용 파일 불러오기
submission.head() #제출 파일이 잘 생성되었는지 확인

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
submission["label"] = pred #예측 값 넣어주기
submission.head() # 데이터가 잘 들어갔는지 확인합니다.

,id,label
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1


In [ ]:
submission.to_csv("./submission.csv",index=False)